# Split Learning with CIFAR-10

In [1]:
#export PYTHONPATH=/home/hroth/Code2/nvflare/splitnn:/home/hroth/Code2/nvflare/splitnn/examples/cifar10

## 1. Download and split the CIFAR-10 dataset
To simulate a vertical split dataset, we first download the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset and distribute it between the two clients.

In [2]:
%env SPLIT_DIR=/tmp/cifar10_vert_splits
%env OVERLAP=10000
!python3 ../pt/utils/cifar10_split_data_vertical.py --split_dir ${SPLIT_DIR} --overlap ${OVERLAP}

env: SPLIT_DIR=/tmp/cifar10_vert_splits
env: OVERLAP=10000
INFO:Cifar10VerticalDataSplitter:[identity=local, run=_]: Partition CIFAR-10 dataset into vertically with 10000 overlapping samples.
Files already downloaded and verified
INFO:Cifar10VerticalDataSplitter:[identity=local, run=_]: save /tmp/cifar10_vert_splits/overlap.npy
INFO:Cifar10VerticalDataSplitter:[identity=local, run=_]: save /tmp/cifar10_vert_splits/site-1.npy
INFO:Cifar10VerticalDataSplitter:[identity=local, run=_]: save /tmp/cifar10_vert_splits/site-2.npy


## 2. Run private set intersection
We are using NVFlare's FL simulator to run the following experiments.

In order to find the overlapping data indices between the different clients participating in split learning, 
we randomly select an subset of the training indices.

In [3]:
import os
#from nvflare import SimulatorRunner
from nvflare.private.fed.app.simulator.simulator_runner import SimulatorRunner

simulator = SimulatorRunner(
    job_folder=f"job_configs/cifar10_psi",
    workspace="/tmp/nvflare/cifar10_psi",
    n_clients=2,
    threads=2
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

ImportError: cannot import name 'SimulatorRunner' from 'nvflare' (unknown location)

The result will be saved on each client's working directory in `intersection.txt`.

We can check the correctness of the result by comparing to the generate ground truth overlap, saved in `overlap.npy`.

### Check the PSI result
We can check the correctness of the result by comparing to the generate ground truth overlap, saved in overlap.npy.

In [ ]:
import os
import numpy as np

split_dir = os.environ["SPLIT_DIR"]
gt_overlap = np.load(os.path.join(split_dir, "overlap.npy"))

psi_overlap_1 = np.loadtxt("/tmp/nvflare/cifar10_psi/simulate_job/site-1/psi/intersection.txt")
psi_overlap_2 = np.loadtxt("/tmp/nvflare/cifar10_psi/simulate_job/site-2/psi/intersection.txt")
                     
print("gt_overlap", gt_overlap, f"n={len(gt_overlap)}")
print("psi_overlap_1", psi_overlap_1, f"n={len(psi_overlap_1)}")
print("psi_overlap_2", psi_overlap_2, f"n={len(psi_overlap_2)}")

intersect_1 = np.intersect1d(psi_overlap_1, gt_overlap, assume_unique=True)
intersect_2 = np.intersect1d(psi_overlap_2, gt_overlap, assume_unique=True)

print(f"Found {100*len(intersect_1)/len(gt_overlap):.1f}% of the overlapping sample ids for site-1.")
print(f"Found {100*len(intersect_2)/len(gt_overlap):.1f}% of the overlapping sample ids for site-2.")

## 3. Run simulated split-learning experiments
Next we use the `intersection.txt` files to align the datasets on each participating site in order to do split learning.
The [config_fed_client.json](./job_configs/cifar10_splitnn/site-1/config/config_fed_client.json) takes as input the previously generated intersection file for each site.
```
    {
        "id": "cifar10-learner",
        "path": "pt.learners.cifar10_learner_splitnn.CIFAR10LearnerSplitNN",
        "args": {
            "dataset_root": "{DATASET_ROOT}",
            "intersection_file": "{INTERSECTION_FILE}",
            "lr": 1e-2,
            "model": {"path": "pt.networks.split_nn.SplitNN", "args":  {"split_id":  0}},
            "timeit": true
        }
    }
```
On the server side, the [config_fed_server.json](./job_configs/cifar10_splitnn/server/config/config_fed_server.json) needs to specify the size of the training dataset in order to generate random sample ids to build each batch during training. Here, the training set size (`train_size`) is equal to the number of overlapping samples defined above.
```
    {
        "id": "splitnn_ctl",
        "path": "pt.workflows.splitnn_workflow.SplitNNController",
        "args": {
            "num_rounds" : "{num_rounds}",
            "batch_size": "{batch_size}",
            "train_size": "{train_size}",
            "start_round": 0,
            "persistor_id": "persistor",
            "task_timeout": 0,
            "shareable_generator_id": "shareable_generator",
            "timeit": true
        }
    }
```
To run the experiment, execute:

In [ ]:
import os
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder=f"job_configs/cifar10_splitnn",
    workspace="/tmp/nvflare/cifar10_splitnn",
    n_clients=2,
    threads=2
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

In [ ]:
#nvflare simulator job_configs/cifar10_splitnn --workspace /tmp/nvflare/splitnn_cifar10 --threads 2 --n_clients 2